Importa as bibliotecas

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import wilcoxon

Ler os dados 

In [2]:
dados = pd.read_excel("results.xlsx")

In [3]:
dados.head()

,Acurácia I,Acurácia B,ROC AUC -I,ROC-AUC-B,PR-AUC-I,PR-AUC-B,Precisão 0 - I,Precisão 0 - B,Recall 0 - I,Recall 0 - B,...,Precisão 1 - B,Recall 1 - I,Recall 1 - B,F1-Score 1 - I,F1-Score 1 - B,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
0,0.797450,0.851064,0.73,0.92,0.02,0.94,0.991827,0.785714,0.802033,0.965426,...,0.955172,0.339474,0.736702,0.032142,0.831832,NaN,NaN,NaN,NaN,NaN
1,0.394749,0.565000,0.56,0.72,0.00,0.70,0.998203,0.842105,0.394213,0.160000,...,0.535912,0.655172,0.970000,0.004426,0.690391,NaN,NaN,NaN,NaN,NaN
2,0.970521,0.775641,0.97,0.77,0.54,0.84,0.998373,0.732323,0.971805,0.895062,...,0.850877,0.844512,0.646667,0.366160,0.734848,NaN,NaN,NaN,NaN,NaN
3,0.770466,0.813406,0.96,0.99,0.40,0.99,0.999468,0.983240,0.769024,0.637681,...,0.731903,0.949275,0.989130,0.062071,0.841294,NaN,NaN,NaN,NaN,Classe 1
4,0.222017,0.565315,0.82,0.95,0.34,0.97,0.998147,0.869048,0.215448,0.162946,...,0.533582,0.955357,0.975000,0.021340,0.689711,NaN,NaN,NaN,NaN,NaN


Elimina as colunas vazias

In [4]:
dados.dropna(axis=1)

,Acurácia I,Acurácia B,ROC AUC -I,ROC-AUC-B,PR-AUC-I,PR-AUC-B,Precisão 0 - I,Precisão 0 - B,Recall 0 - I,Recall 0 - B,F1-Score 0 - I,F1-Score 0 - B,Precisão 1 - I,Precisão 1 - B,Recall 1 - I,Recall 1 - B,F1-Score 1 - I,F1-Score 1 - B
0,0.797450,0.851064,0.73,0.92,0.02,0.94,0.991827,0.785714,0.802033,0.965426,0.886889,0.866348,0.016869,0.955172,0.339474,0.736702,0.032142,0.831832
1,0.394749,0.565000,0.56,0.72,0.00,0.70,0.998203,0.842105,0.394213,0.160000,0.565211,0.268908,0.002221,0.535912,0.655172,0.970000,0.004426,0.690391
2,0.970521,0.775641,0.97,0.77,0.54,0.84,0.998373,0.732323,0.971805,0.895062,0.984910,0.805556,0.233755,0.850877,0.844512,0.646667,0.366160,0.734848
3,0.770466,0.813406,0.96,0.99,0.40,0.99,0.999468,0.983240,0.769024,0.637681,0.869232,0.773626,0.032084,0.731903,0.949275,0.989130,0.062071,0.841294
4,0.222017,0.565315,0.82,0.95,0.34,0.97,0.998147,0.869048,0.215448,0.162946,0.354400,0.274436,0.010790,0.533582,0.955357,0.975000,0.021340,0.689711
5,0.940778,0.546334,0.55,0.60,0.04,0.63,0.973239,0.585321,0.965594,0.324187,0.969401,0.417266,0.073927,0.531360,0.093750,0.769388,0.082667,0.628595
6,0.851493,0.548113,0.73,0.70,0.05,0.68,0.986916,0.699301,0.859970,0.186567,0.919081,0.294551,0.055861,0.524537,0.420863,0.917939,0.098630,0.667592
7,0.726746,0.213889,0.22,0.15,0.00,0.29,0.991174,0.132743,0.731381,0.075000,0.841687,0.095847,0.001337,0.251012,0.052326,0.387500,0.002607,0.304668
8,0.954097,0.601622,0.77,0.72,0.11,0.73,0.960458,0.540210,0.993083,0.906690,0.976498,0.677038,0.066852,0.810714,0.011976,0.341182,0.020313,0.480254
9,0.958472,0.718750,0.58,0.76,0.01,0.79,0.995526,0.669565,0.962605,0.916667,0.978789,0.773869,0.008631,0.844444,0.070000,0.500000,0.015368,0.628099


In [7]:
#escolhe os dados para fazer o teste de normalidade e transforma em vetor
data0 = np.array(dados['Acurácia I'])
data1 = np.array(dados['Acurácia B'])
data2 = np.array(dados['ROC AUC -I'])
data3 = np.array(dados['ROC-AUC-B'])
data4 = np.array(dados['PR-AUC-I'])
data5 = np.array(dados['PR-AUC-B'])

In [8]:
# Realiza o teste Shapiro-Wilk para normalidade
statistic, p_value = stats.shapiro(data0)

# Indica o nível de significância 
alpha = 0.05

# Interpreta os resultados
if p_value > alpha:
    print("Os dados estão normalmente distrbuídos (não rejeita a hipótese nula).")
else:
    print("Os dados não estão normalmente distribuídos (rejeita a hipótese nula).")

print(p_value)

Os dados não estão normalmente distribuídos (rejeita a hipótese nula).
3.5860906791640446e-05
